In [1]:
DISTANCE_AIRPORT_CITYCENTER_KM = 15

In [2]:
import pandas as pd


## 1.Export

### Read airport data from CSV

In [3]:
airport_df = pd.read_csv("airports.csv")

### Import cities_df

In [ ]:
import sqlalchemy
sys.path.append('..\keys')
import key
schema="gans_db"   # name of the database you want to use here
host="localhost"        # to connect to your local server
user="root"
password=key.SQL_PASSWORD # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
city_df = pd.read_sql_table('city',con=con)

## 2.Transform

### Clean airport_df

####  drop unneeded columns

In [4]:
airport_df=airport_df.drop(["scheduled_service","gps_code","iata_code","local_code","home_link","wikipedia_link","keywords"],axis=1)

####  rename some columns

In [5]:
airport_df = airport_df.rename(columns={"ident":"icao"})

####  filter for large/meduim airport

In [6]:
airport_df.type.unique()

array(['heliport', 'small_airport', 'closed', 'seaplane_base',
       'balloonport', 'medium_airport', 'large_airport'], dtype=object)

In [7]:
airport_df=airport_df.loc[~airport_df.type.isin(["heliport","closed","seaplane_base","balloonport","small_airport"])]

In [9]:
airport_df

,id,icao,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality
6401,12243,5A8,medium_airport,Aleknagik / New Airport,59.282600,-158.617996,66.0,NaN,US,US-AK,Aleknagik
10856,332240,AF-0005,medium_airport,Khost International Airport (U.C.),33.284605,69.807340,4204.0,AS,AF,AF-KHO,Khost
10890,3,AGGH,large_airport,Honiara International Airport,-9.428000,160.054993,28.0,OC,SB,SB-CT,Honiara
10895,4,AGGM,medium_airport,Munda Airport,-8.327970,157.263000,10.0,OC,SB,SB-WE,Munda
10917,312371,AHJ,medium_airport,Hongyuan Airport,32.531540,102.352240,11600.0,AS,CN,CN-51,Aba
...,...,...,...,...,...,...,...,...,...,...,...
68322,307013,ZYTN,medium_airport,Tonghua Sanyuanpu Airport,42.253889,125.703333,1200.0,AS,CN,CN-22,Tonghua
68323,27243,ZYTX,large_airport,Shenyang Taoxian International Airport,41.639801,123.483002,198.0,AS,CN,CN-21,"Hunnan, Shenyang"
68325,27244,ZYYJ,medium_airport,Yanji Chaoyangchuan Airport,42.882801,129.451004,624.0,AS,CN,CN-22,Yanji
68326,317861,ZYYK,medium_airport,Yingkou Lanqi Airport,40.542524,122.358600,NaN,AS,CN,CN-21,"Laobian, Yingkou"


####  filter for airports within 30km from cities in city Table

In [10]:

# Python 3 program to calculate Distance Between Two Points on Earth
from math import radians, cos, sin, asin, sqrt
def distance_km(airport_row,lat2,lon2,city_id):
     
    # The math module contains a function named
  
    #row contains location of airport
    lat1=airport_row.latitude_deg
    lon1=airport_row.longitude_deg
    
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
    
    dist_km = c * r
    
    if dist_km <= DISTANCE_AIRPORT_CITYCENTER_KM :
        ret = city_id
    else:
        ret = 0
    # calculate the result
    return ret

In [11]:
cities_df.shape[0]

36

In [12]:
df

,id,icao,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality
6401,12243,5A8,medium_airport,Aleknagik / New Airport,59.282600,-158.617996,66.0,NaN,US,US-AK,Aleknagik
10856,332240,AF-0005,medium_airport,Khost International Airport (U.C.),33.284605,69.807340,4204.0,AS,AF,AF-KHO,Khost
10890,3,AGGH,large_airport,Honiara International Airport,-9.428000,160.054993,28.0,OC,SB,SB-CT,Honiara
10895,4,AGGM,medium_airport,Munda Airport,-8.327970,157.263000,10.0,OC,SB,SB-WE,Munda
10917,312371,AHJ,medium_airport,Hongyuan Airport,32.531540,102.352240,11600.0,AS,CN,CN-51,Aba
...,...,...,...,...,...,...,...,...,...,...,...
68322,307013,ZYTN,medium_airport,Tonghua Sanyuanpu Airport,42.253889,125.703333,1200.0,AS,CN,CN-22,Tonghua
68323,27243,ZYTX,large_airport,Shenyang Taoxian International Airport,41.639801,123.483002,198.0,AS,CN,CN-21,"Hunnan, Shenyang"
68325,27244,ZYYJ,medium_airport,Yanji Chaoyangchuan Airport,42.882801,129.451004,624.0,AS,CN,CN-22,Yanji
68326,317861,ZYYK,medium_airport,Yingkou Lanqi Airport,40.542524,122.358600,NaN,AS,CN,CN-21,"Laobian, Yingkou"


In [13]:
df["city_id"]= 0
for i in range(cities_df.shape[0]):
    dist_accepted=df.apply(distance_km,axis=1,result_type="expand",args=(cities_df.lat[i],cities_df.lon[i],cities_df.city_id[i]))
    # if(df.iloc[i].airp_city_id != 0):
    #     print("Attention, One Airport related to more than one City!!")
    #     print(dist_accepted.iloc[i])
        
    df["city_id"] += dist_accepted

In [14]:
df.city_id.unique()


array([ 0,  9, 15, 29,  3, 14, 23, 18,  6, 11, 21,  8, 24, 10,  1, 17, 31,
       25, 30, 27, 34, 32, 26, 20,  7, 22], dtype=int64)

In [15]:
airport_df = df.loc[df.city_id > 0]

In [16]:
airport_df.shape

(35, 12)

In [17]:
airport_df

,id,icao,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,city_id
12572,44941,AZ-0001,medium_airport,Zabrat Airport,40.495542,49.976807,36.0,AS,AZ,AZ-BA,Baku,9
20249,2214,EDDH,large_airport,Hamburg Helmut Schmidt Airport,53.630402,9.988230,53.0,EU,DE,DE-HH,Hamburg,15
20325,2234,EDHI,medium_airport,Hamburg-Finkenwerder Airport,53.535278,9.835556,23.0,EU,DE,DE-HH,Hamburg,15
20810,2389,EGBB,large_airport,Birmingham International Airport,52.453899,-1.748030,327.0,EU,GB,GB-ENG,Birmingham,29
20922,2430,EGLC,medium_airport,London City Airport,51.505299,0.055278,19.0,EU,GB,GB-ENG,London,3
21551,2637,EPWA,large_airport,Warsaw Chopin Airport,52.165699,20.967100,362.0,EU,PL,PL-MZ,Warsaw,14
36871,3977,LBSF,large_airport,Sofia Airport,42.696693,23.411436,1742.0,EU,BG,BG-23,Sofia,23
36947,4004,LEBL,large_airport,Josep Tarradellas Barcelona-El Prat Airport,41.297100,2.078460,12.0,EU,ES,ES-CT,Barcelona,18
36960,4044,LECU,medium_airport,Madrid-Cuatro Vientos Airport,40.370701,-3.785140,2269.0,EU,ES,ES-M,Madrid,6
36980,4012,LEGT,medium_airport,Getafe Air Base,40.294102,-3.723830,2031.0,EU,ES,ES-M,Getafe,6


In [18]:
airport_df=airport_df[["city_id","icao","name","type"]]

In [19]:
airport_df

,city_id,icao,name,type
12572,9,AZ-0001,Zabrat Airport,medium_airport
20249,15,EDDH,Hamburg Helmut Schmidt Airport,large_airport
20325,15,EDHI,Hamburg-Finkenwerder Airport,medium_airport
20810,29,EGBB,Birmingham International Airport,large_airport
20922,3,EGLC,London City Airport,medium_airport
21551,14,EPWA,Warsaw Chopin Airport,large_airport
36871,23,LBSF,Sofia Airport,large_airport
36947,18,LEBL,Josep Tarradellas Barcelona-El Prat Airport,large_airport
36960,6,LECU,Madrid-Cuatro Vientos Airport,medium_airport
36980,6,LEGT,Getafe Air Base,medium_airport


In [20]:
airport_df.sort_values(by="city_id")

,city_id,icao,name,type
38584,1,LTBA,İstanbul Atatürk Airport,large_airport
20922,3,EGLC,London City Airport,medium_airport
36960,6,LECU,Madrid-Cuatro Vientos Airport,medium_airport
36980,6,LEGT,Getafe Air Base,medium_airport
57394,7,UKKK,Kiev Zhuliany International Airport,medium_airport
37838,8,LIRU,Rome Urbe Airport,medium_airport
37823,8,LIRA,Ciampino–G. B. Pastine International Airport,medium_airport
12572,9,AZ-0001,Zabrat Airport,medium_airport
57129,9,UBBB,Heydar Aliyev International Airport,large_airport
38322,10,LRBS,Băneasa International Airport,medium_airport


## 3.Load

### Connect with AWS

In [21]:
import sqlalchemy as db
sys.path.append('..\keys')
import key

schema="gans_db"   # name of the database you want to use here
host="wbs-gans-db.c87binzvwbjx.eu-central-1.rds.amazonaws.com"        # to connect to your local server
user="admin"
password=key.SQL_PASSWORD # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

### Export Final df

In [20]:
airport_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 12572 to 63453
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   city_id  35 non-null     int64 
 1   icao     35 non-null     object
 2   name     35 non-null     object
 3   type     35 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.4+ KB


In [22]:
airport_df.to_sql('airport',con=con,if_exists='append',index=False)

35